In [2]:
!cd venv/Scripts/

In [3]:
!activate


D:\ai_training>() 


In [4]:
!pwd

/cygdrive/d/ai_training


In [5]:
!pip list

Package                   Version
------------------------- --------------------
absl-py                   2.1.0
anyio                     4.3.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     23.2.0
Babel                     2.15.0
beautifulsoup4            4.12.3
bleach                    6.1.0
boto3                     1.34.103
botocore                  1.34.103
certifi                   2023.7.22
cffi                      1.16.0
chardet                   4.0.0
charset-normalizer        3.3.2
click                     8.1.7
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.2.1
cycler                    0.10.0
debugpy                   1.8.1
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.0.1
fastdtw                   0.3.4
fastjsonschema            2.19.1
filel

In [6]:
import cv2
import torch
import ultralytics
from ultralytics import YOLO
from scipy.stats import wasserstein_distance
from fastdtw import fastdtw
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from collections import defaultdict
from sklearn.decomposition import PCA
from skimage.feature import hog


# Function to extract frames from a video
def generate_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame to 640x640
        resized_frame = cv2.resize(frame, (640, 640))
        # Convert frame to RGB as YOLO expects RGB images
        rgb_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
        yield rgb_frame
    cap.release()

# Load the YOLOv8 model
model = YOLO('best.pt')
# print(dir(ultralytics))
# print(dir(model))
# Function to get features from frames using the YOLOv8 model
def get_features(frame_generator):
    prev_frame = next(frame_generator)  # Get the first frame from the generator
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_RGB2GRAY)
    features = []  # This will be a list of arrays

    for frame in frame_generator:
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

        # Compute optical flow for the whole frame
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.1, 0)
        flow_magnitude, flow_angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        global_flow_features = [flow_magnitude.mean(), flow_angle.mean()]  # Global flow features

        # Compute HOG features for the whole frame
        global_hog_features = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)

        # Run YOLOv8 model to detect objects
        results = model(frame)
        if results[0].boxes.data.shape[0] > 0:
            bboxes = results[0].boxes.xyxy.cpu().numpy()

            frame_features = []
            for bbox in bboxes:
                # x1, y1, x2, y2 = map(int, bbox[:4])
                # Combine bounding box coordinates with global features
                combined_features = np.hstack((bbox, global_hog_features, global_flow_features))
                frame_features.append(combined_features)

            # Append each frame's features to the list; handle varying number of detections
            features.append(np.array(frame_features))
        else:
            features.append(np.array([]))

        prev_gray = gray  # Update the previous frame for optical flow calculation

    returdef calculate_mfr(features):
    # Check if the features array is empty
    if features.size == 0:
        return np.array([])  # Return empty array if no features

    # Calculate the mean of the features across all samples (rows)
    mean_features = np.mean(features, axis=0)
    return mean_features


def preprocess_features_for_dtw(boxes):
    # Access the raw tensor data from the 'Boxes' object
    if hasattr(boxes, 'data'):
        box_data = boxes.data
    else:
        raise ValueError("Boxes object does not have 'data' attribute")

    # Check if the box data tensor is empty
    if box_data.numel() == 0:
        return np.array([])  # Return an empty array if no boxes

    # Calculate centers, areas, and aspect ratios from the tensor
    centers_x = (box_data[:, 0] + box_data[:, 2]) / 2
    centers_y = (box_data[:, 1] + box_data[:, 3]) / 2
    widths = box_data[:, 2] - box_data[:, 0]
    heights = box_data[:, 3] - box_data[:, 1]
    areas = widths * heights
    aspect_ratios = widths / heights

    # Combine the features into a single tensor before converting to NumPy
    features = torch.stack((centers_x, centers_y, areas, aspect_ratios), dim=1)

    # Convert the tensor to NumPy array after moving it to CPU
    features_array = features.cpu().numpy()

    return features_array

def calculate_mfr(features):
    # Flatten all feature arrays into a single list
    all_features = [item for sublist in features for item in sublist if item.size > 0]
    if not all_features:
        return np.array([])  # Return empty array if no features

    # Stack all features into a single NumPy array and calculate the mean
    all_features = np.vstack(all_features)
    mean_features = np.mean(all_features, axis=0)
    return mean_features

def apply_pca_to_features(features, sampling_interval=30, variance_threshold=0.50):
    # Flatten the feature sets and ensure they're sampled at a reasonable rate
    sampled_features = [f for feature_set in features for f in feature_set if f.size > 0][::sampling_interval]
    if not sampled_features:
        print("No features available for PCA.")
        return None

    # Stack features into a matrix
    feature_matrix = np.vstack(sampled_features)

    # Initialize PCA with the variance threshold
    pca = PCA(n_components=variance_threshold)  # Adjust n_components to capture the specified variance

    # Apply PCA
    return pca.fit_transform(feature_matrix)


# Function to calculate the Earth Mover's Distance (EMD)
def calculate_emd(features1, features2):
    # Check if either distribution is empty
    if features1.size == 0 or features2.size == 0:
        # Return a high value to indicate no comparison possible
        return float('inf')

    # Calculate and return the Earth Mover's Distance
    return wasserstein_distance(features1, features2)

def calculate_mte(features1, features2):
    # Calculate DTW distance
    distance, path = fastdtw(features1, features2)
    return distance

def calculate_emd_and_mte(f1, f2):
    emd = calculate_emd(f1, f2)
    mte = calculate_mte(f1, f2)
    return emd, mte

# Function to process all frames using multithreading
def process_frames(video1_features, video2_features):
    emd_scores = defaultdict(list)  # Use defaultdict for easier structure
    mte_scores = defaultdict(list) 
    total_tasks = len(video1_features) * len(video2_features)
    completed_tasks = 0

    with ThreadPoolExecutor(max_workers=None) as executor:
        # Submit tasks without immediately waiting for results 
        futures = {executor.submit(calculate_emd_and_mte, f1, f2): (i, j)
                   for i, f1 in enumerate(video1_features) 
                   for j, f2 in enumerate(video2_features)}

        # Process results as they become available
        for future in as_completed(futures):
            i, j = futures[future]
            try:
                emd, mte = future.result()  # Get result when ready
                emd_scores[i].append(emd)
                mte_scores[i].append(mte)
                completed_tasks += 1

                if completed_tasks % 10 == 0:
                    print(f"Progress: {completed_tasks / total_tasks * 100:.2f}% completed") 

            except Exception as exc:
                print(f'Task generated an exception: {exc}') 

    # Convert defaultdict to list of lists (if needed)
    # emd_scores = [emd_scores[i] for i in range(len(video1_features))]
    # mte_scores = [mte_scores[i] for i in range(len(video1_features))]

    return emd_scores, mte_scores

In [7]:
def apply_pca_to_features(features, sampling_interval=30):
    # Flatten the feature sets and ensure they're sampled at a reasonable rate
    sampled_features = [f for feature_set in features for f in feature_set if f.size > 0][::sampling_interval]
    if not sampled_features:
        print("No features available for PCA.")
        return None

    # Stack features into a matrix
    feature_matrix = np.vstack(sampled_features)

    # Initialize PCA with the variance threshold
    pca = PCA(n_components=int(len(feature_matrix)/2))  # Adjust n_components to capture the specified variance

    # Apply PCA
    return pca.fit_transform(feature_matrix)

In [8]:
# Extract frames from both videos
video1_frame_gen = generate_frames('IC1-08nov-TGI-Hazel-Jennie-Spring.mp4')
video2_frame_gen = generate_frames('IC3-TGO-08nov-Hazel-Georgia-Spring.mp4')
video1_features = get_features(video1_frame_gen)
video2_features = get_features(video2_frame_gen)



0: 640x640 (no detections), 7.5ms
Speed: 4.0ms preprocess, 7.5ms inference, 1285.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.3ms
Speed: 2.1ms preprocess, 7.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.0ms
Speed: 2.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.0ms
Speed: 2.0ms preprocess, 7.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.7ms
Speed: 2.5ms preprocess, 7.7ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 13.5ms
Speed: 3.5ms preprocess, 13.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.0ms
Speed: 3.5ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.1ms
Speed: 2.5ms preprocess, 8.1ms inference, 1

In [9]:
def calculate_mfr(features):
    # Check if the features array is empty
    if features.size == 0:
        return np.array([])  # Return empty array if no features

    # Calculate the mean of the features across all samples (rows)
    mean_features = np.mean(features, axis=0)
    return mean_features
    

In [11]:
video1_pca = apply_pca_to_features(video1_features)
video2_pca = apply_pca_to_features(video2_features)

In [12]:
print(video1_pca)

[[    -218.97     -32.644     -8.2424      1.2183     -9.5549]
 [    -177.03     -31.816     -10.835     -1.8965     -4.5085]
 [    -133.57     -26.817      14.465     -5.6157     -10.429]
 [    -111.12     -27.456      15.991     -8.7062     -2.9866]
 [     381.46     -126.38     -13.882     -12.684       6.696]
 [    -80.848     -59.094      10.492     -8.5163      12.249]
 [    -103.73      27.794       7.312      2.4839      4.5265]
 [     195.19      160.38      41.365      8.2566      3.1007]
 [     374.43     -113.46     -6.0037      13.083     -10.096]
 [     82.835      294.91     -32.619     -4.4835    -0.82064]
 [    -208.64     -65.421     -18.041       16.86      11.823]]


In [16]:
print(emd_scores)

[[50.748757017459184, 56.967048345142565, 50.49849952571025, 55.289637768961335, 53.38456977303421, 51.53465509166278, 57.63626590491933, 51.10127803887211, 98.96809641714505, 51.11715953469398, 55.98483987217032, 51.61230736926133, 51.84996378551565, 54.18597307348319, 54.27014218877592, 49.4476829275668, 53.25223000893573, 53.1658140923625, 51.857029189659706, 50.84729032760098, 50.21733230399632, 49.74922869645671, 49.684263446753874, 48.109190691348786, 50.50266453097629, 51.99182050695613, 51.52031225854704, 51.10511374779534, 51.13523356204074, 53.46875201810913, 52.82069120290756, 52.40146073887894], [42.32788257034932, 48.5461738980327, 42.07762507860039, 46.86876332185147, 44.963695325924334, 43.11378064455291, 49.215391457809474, 42.680403591762236, 90.54722197003518, 42.69628508758411, 47.563965425060445, 43.191432922151456, 43.42908933840579, 45.765098626373316, 45.84926774166605, 41.02680848045694, 44.83135556182586, 44.74493964525264, 43.436154742549824, 42.42641588049111

In [17]:
if video1_pca is not None and video2_pca is not None:
    mfr_video1 = calculate_mfr(video1_pca)
    mfr_video2 = calculate_mfr(video2_pca)
    print("MFR Video 1:", mfr_video1)
    print("MFR Video 2:", mfr_video2)

# Outputs the EMD and MTE scores
print("EMD Scores:", emd_scores)
print("MTE Scores:", mte_scores)

MFR Video 1: [-3.3589e-14  5.6843e-14 -4.1018e-14 -1.9378e-15 -4.6831e-15]
MFR Video 2: [-3.0198e-14  1.8652e-14           0 -1.7694e-16  -1.249e-16 -1.3878e-16 -2.3592e-16 -1.1102e-16   -1.36e-15 -2.7756e-15  8.1185e-16 -3.6082e-16  4.9266e-16  1.5266e-16 -4.3021e-16  5.9588e-16]
EMD Scores: [[50.748757017459184, 56.967048345142565, 50.49849952571025, 55.289637768961335, 53.38456977303421, 51.53465509166278, 57.63626590491933, 51.10127803887211, 98.96809641714505, 51.11715953469398, 55.98483987217032, 51.61230736926133, 51.84996378551565, 54.18597307348319, 54.27014218877592, 49.4476829275668, 53.25223000893573, 53.1658140923625, 51.857029189659706, 50.84729032760098, 50.21733230399632, 49.74922869645671, 49.684263446753874, 48.109190691348786, 50.50266453097629, 51.99182050695613, 51.52031225854704, 51.10511374779534, 51.13523356204074, 53.46875201810913, 52.82069120290756, 52.40146073887894], [42.32788257034932, 48.5461738980327, 42.07762507860039, 46.86876332185147, 44.963695325924

In [48]:
from sklearn.preprocessing import StandardScaler

def prepare_features(features):
    """ Flatten and standardize features for PCA. """
    # Flatten the feature sets
    # all_features = np.vstack([f for feature_set in features for f in feature_set if f.size > 0])
    
    # Standardize features
    scaler = StandardScaler()
    standardized_features = scaler.fit_transform(all_features)
    return standardized_features

def apply_pca_to_features(features, sampling_interval=10):
    # Flatten the feature sets and ensure they're sampled at a reasonable rate
    sampled_features = [f for feature_set in features for f in feature_set if f.size > 0]
    if not sampled_features:
        print("No features available for PCA.")
        return None

    # Stack features into a matrix
    feature_matrix = np.vstack(sampled_features)

    # Initialize PCA with the variance threshold
    pca = PCA(n_components=31)  # Adjust n_components to capture the specified variance

    # Apply PCA
    return pca.fit_transform(feature_matrix)

In [49]:
# Assuming you've already prepared your features as arrays


# Determine the maximum number of components based on the available data

# Apply PCA to both sets of features using a safe number of components
video1_pca = apply_pca_to_features(video1_features)
video2_pca = apply_pca_to_features(video2_features)
# Initialize lists to store the similarity scores
emd_scores = []
mte_scores = []

In [20]:
def calculate_similarity_matrix(features1, features2):
    num_frames1 = len(features1)
    num_frames2 = len(features2)
    similarity_matrix = np.zeros((num_frames1, num_frames2))

    for i in range(num_frames1):
        for j in range(num_frames2):
            # Calculate similarity, e.g., using negative Euclidean distance
            if features1[i].size == 0 or features2[j].size == 0:
                similarity_matrix[i, j] = float('inf')
            else:
                feature1 = np.concatenate(features1[i], axis=None)
                feature2 = np.concatenate(features2[j], axis=None)
                similarity_matrix[i, j] = -np.linalg.norm(feature1 - feature2)

    return similarity_matrix

In [21]:
def synchronize_videos(similarity_matrix):
    # Compute the DTW path based on the similarity matrix
    path = []
    num_frames1, num_frames2 = similarity_matrix.shape
    
    distance, path = fastdtw(np.arange(num_frames1), np.arange(num_frames2), dist=lambda x, y: similarity_matrix[int(x), int(y)])
    
    return path

In [55]:
similarity_matrix = calculate_similarity_matrix(video1_pca, video2_pca)
print(similarity_matrix)
path = synchronize_videos(similarity_matrix)

print("Synchronization Path:", path)

[[    -203.57     -205.53     -204.35 ...     -214.12      -212.6     -214.13]
 [    -203.17     -205.11     -203.99 ...     -213.59     -212.06      -213.6]
 [     -205.1     -207.04     -205.93 ...     -215.42     -213.89     -215.43]
 ...
 [    -201.31     -203.44     -202.13 ...     -211.93     -210.32        -212]
 [    -189.62     -191.57     -190.71 ...     -199.02     -197.37     -199.05]
 [    -222.78     -224.86     -223.09 ...     -236.07     -234.65     -236.11]]
Synchronization Path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (0, 31), (0, 32), (0, 33), (0, 34), (0, 35), (0, 36), (0, 37), (0, 38), (0, 39), (0, 40), (0, 41), (0, 42), (0, 43), (0, 44), (0, 45), (0, 46), (0, 47), (0, 48), (0, 49), (0, 50), (0, 51), (0, 52), (0, 53), (0, 54), (0, 55), (0,

In [51]:
def synchronize_videos(similarity_matrix):
    # Compute the DTW path based on the similarity matrix
    path = []
    num_frames1, num_frames2 = similarity_matrix.shape
    
    distance, path = fastdtw(np.arange(num_frames1), np.arange(num_frames2), dist=lambda x, y: similarity_matrix[int(x), int(y)])
    
    return path

In [52]:
path = synchronize_videos(similarity_matrix)

print("Synchronization Path:", path)

Synchronization Path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (0, 31), (0, 32), (0, 33), (0, 34), (0, 35), (0, 36), (0, 37), (0, 38), (0, 39), (0, 40), (0, 41), (0, 42), (0, 43), (0, 44), (0, 45), (0, 46), (0, 47), (0, 48), (0, 49), (0, 50), (0, 51), (0, 52), (0, 53), (0, 54), (0, 55), (0, 56), (0, 57), (0, 58), (0, 59), (0, 60), (0, 61), (0, 62), (0, 63), (0, 64), (0, 65), (0, 66), (0, 67), (0, 68), (0, 69), (0, 70), (0, 71), (0, 72), (0, 73), (0, 74), (0, 75), (0, 76), (0, 77), (0, 78), (0, 79), (0, 80), (0, 81), (0, 82), (0, 83), (0, 84), (0, 85), (0, 86), (0, 87), (0, 88), (0, 89), (0, 90), (0, 91), (0, 92), (0, 93), (0, 94), (0, 95), (0, 96), (0, 97), (0, 98), (0, 99), (0, 100), (0, 101), (0, 102), (0, 103), (0, 104), (0, 105), (0, 106), (0, 107), (0, 108

In [56]:
def calculate_emd(features1, features2, path):
    emd_score = 0
    for i, j in path:
        frame1_features = features1[i]
        frame2_features = features2[j]
        # Assuming Euclidean distance for simplicity
        emd_score += np.linalg.norm(frame1_features - frame2_features)
    return emd_score / len(path)

def calculate_mfr(path, total_frames, tolerance=2):
    matched_frames = 0
    last_index = -tolerance - 1
    for i, j in path:
        if i > last_index + tolerance:
            matched_frames += 1
            last_index = i
    return matched_frames / total_frames * 100

def calculate_mte(path):
    total_error = 0
    for i, j in path:
        total_error += abs(i - j)
    return total_error / len(path)
    

In [59]:
emd_score = calculate_emd(video1_pca, video2_pca, path)
mfr_score1 = calculate_mfr(path, len(video1_pca))
mfr_score2 = calculate_mfr(path, len(video2_pca))
mte_score = calculate_mte(path)

In [60]:
print("Matched Frame Rate for video1:", mfr_score1)
print("Matched Frame Rate for video2:", mfr_score2)
print("EMD Score:", emd_score)
print("Mean Temporal Error:", mte_score)

Matched Frame Rate for video1: 33.55048859934853
Matched Frame Rate for video2: 10.934182590233545
EMD Score: 391.0649549406016
Mean Temporal Error: 438.37339743589746
